In [2]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [3]:
# load data
df = pd.read_pickle("./df_2020.pkl")
df

,Headline,Date,category,Word count,Article: No Stopwords,Lemmas,BoW
0,Ukrainischer Kleriker warnt vor den Schweinere...,2020-09-28 06:45:00,gesellschaft,8,"[Ukrainischer, Kleriker, warnt, Schweinereien,...","[Ukrainischer, Kleriker, warnen, Schweinerei, ...","{'Ukrainischer': 1, 'Kleriker': 1, 'warnen': 1..."
1,Der MagnetKünstler Kleben und kleben lassen Video,2020-09-27 19:55:00,gesellschaft,7,"[MagnetKünstler, Kleben, kleben, lassen, Video]","[MagnetKünstler, Kleben, kleben, lassen, Video]","{'MagnetKünstler': 1, 'Kleben': 1, 'kleben': 1..."
2,Es grünt Guccis GärtnerJeans mit Grasflecken f...,2020-09-26 16:55:00,gesellschaft,15,"[grünt, Guccis, GärtnerJeans, Grasflecken, kna...","[grünt, Guccis, GärtnerJeans, Grasflecken, kna...","{'grünt': 1, 'Guccis': 1, 'GärtnerJeans': 1, '..."
3,Chef des Weltwirtschaftsforums Neoliberalismus...,2020-09-26 16:20:00,gesellschaft,8,"[Chef, Weltwirtschaftsforums, Neoliberalismus,...","[Chef, Weltwirtschaftsforums, Neoliberalismus,...","{'Chef': 1, 'Weltwirtschaftsforums': 1, 'Neoli..."
4,Umstrittene Maßnahme Absichtliche CoronaInfekt...,2020-09-26 09:41:00,gesellschaft,9,"[Umstrittene, Maßnahme, Absichtliche, CoronaIn...","[Umstrittene, Maßnahme, Absichtliche, CoronaIn...","{'Umstrittene': 1, 'Maßnahme': 1, 'Absichtlich..."
...,...,...,...,...,...,...,...
8561,Nordmazedonien und Griechenland erhalten Gas a...,2020-01-07 06:30:00,wirtschaft,10,"[Nordmazedonien, Griechenland, erhalten, Gas, ...","[Nordmazedonien, Griechenland, erhalten, Gas, ...","{'Nordmazedonien': 1, 'Griechenland': 1, 'erha..."
8562,Paris Finanzminister warnt USA vor Vergeltung ...,2020-01-06 09:52:00,wirtschaft,11,"[Paris, Finanzminister, warnt, USA, Vergeltung...","[Paris, Finanzminister, warnen, USA, Vergeltun...","{'Paris': 1, 'Finanzminister': 1, 'warnen': 1,..."
8563,USMordanschlag auf iranischen General setzt Fi...,2020-01-03 21:34:00,wirtschaft,8,"[USMordanschlag, iranischen, General, setzt, F...","[USMordanschlag, iranisch, General, setzen, Fi...","{'USMordanschlag': 1, 'iranisch': 1, 'General'..."
8564,Indien wird Deutschland als viertgrößte Volksw...,2020-01-03 06:15:00,wirtschaft,11,"[Indien, Deutschland, viertgrößte, Volkswirtsc...","[Indien, Deutschland, viertgrößte, Volkswirtsc...","{'Indien': 1, 'Deutschland': 1, 'viertgrößte':..."


In [4]:
#preprocess sentences 
def pre_process(df_data):
    df_data = [word.lower() for word in df_data] #lower
    for word in df_data:
        if word == 'video':  # remove 'video'
            df_data.remove(word)
    return df_data

# apply function
df['Article: No Stopwords'] = df['Article: No Stopwords'].apply(lambda row: pre_process(row))
       
# convert into list of lists of tokens 
tokens = df['Article: No Stopwords'].to_list()


In [5]:
# generate own word vector representations
num_features = 300 # number of dimensions to represent the word vector
min_word_count = 3 # large corpus = increse min word count to be considered
num_workers = 2 # number of CPU cores to be used
window_size = 6 # context window size
subsampling = 1e-3 # subsampling rate of frequent terms


# train
model = Word2Vec(tokens, workers=num_workers, size=num_features, min_count=min_word_count, 
                 window=window_size, 
                 sample=subsampling)


# init_sims freezes model, discards unnecessary, uneeded output weights
# only stores hidden layer weights
model.init_sims(replace=True)

# save model
model_name = "RT_News_model"
model.save(model_name)


In [6]:
# load saved domain-specific word2vec model
model_name = "RT_News_model"
model = Word2Vec.load(model_name)


In [7]:
# show most similar words to a word
model.most_similar(positive=['italien'], topn=10)

<ipython-input-7-5a25c1854c2b>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['italien'], topn=10)


[('russland', 0.851146936416626),
 ('usa', 0.8505241870880127),
 ('polizei', 0.8380656242370605),
 ('mehr', 0.8371245265007019),
 ('wegen', 0.8310774564743042),
 ('coronakrise', 0.8260030150413513),
 ('ab', 0.8243602514266968),
 ('millionen', 0.8232160806655884),
 ('china', 0.8230593800544739),
 ('neue', 0.8106922507286072)]

In [8]:
# which word does not match
model.doesnt_match("merkel putin trump virus".split())

<ipython-input-8-09043c860f7e>:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  model.doesnt_match("merkel putin trump virus".split())
/home/tina/anaconda3/lib/python3.8/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'virus'

In [9]:
# show similarity of two terms
model.similarity('merkel', 'regierung')

<ipython-input-9-cf58ee20fc65>:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  model.similarity('merkel', 'regierung')


0.76613784